<a href="https://colab.research.google.com/github/shawn-kg/DataMining/blob/main/assign4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Assign 4: Linear Regression

## Data Preprocessing

In [3]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler(with_mean=True, with_std=True)
data = pd.read_csv('./OnlineNewsPopularity.csv')

data = data.drop(data.columns[[0,1,4,5,6,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38]], axis=1)
# print(data.shape)
data = data.to_numpy()

np.random.seed(42)
np.random.shuffle(data)
target = data[:,-1]
data = np.delete(data,-1,1)
# print(data)
train = data[:31715]
# print(train)
valid = data[31715:31715+3964]
# print(valid.shape)
test = data[31715+3964:31715+3964+3965]
# print(test.shape)
train_means = np.mean(train, axis=0)
# print(train_means)
train_std = np.std(train, axis=0)
# scaler.fit(train)
# train = scaler.transform(train)
# print(train_std)
for i in range(31715):
  for j in range(29):
    train[i][j] = (train[i][j] - train_means[j])/train_std[j]
for i in range(3964):
  for j in range(29):
    valid[i][j] = (valid[i][j] - train_means[j])/train_std[j]
for i in range(3965):
  for j in range(29):
    test[i][j] = (test[i][j] - train_means[j])/train_std[j]
train = np.insert(train,0,np.ones((31715)),axis=1)
valid = np.insert(valid,0,np.ones((3964)),axis=1)
test = np.insert(test,0,np.ones((3965)),axis=1)
print("Train")
print(train)
print("Valid")
print(valid)
print("Test")
print(test)

Train
[[ 1.          0.76066508  1.00690312 ... -0.26330178  0.83711369
  -0.69029101]
 [ 1.         -0.65736617 -0.08776171 ... -0.26330178  0.83711369
  -0.69029101]
 [ 1.         -0.65736617 -0.66716439 ... -0.26330178  0.83711369
  -0.69029101]
 ...
 [ 1.         -0.65736617  0.90855432 ... -0.26330178  0.83711369
  -0.69029101]
 [ 1.          0.28798799  3.06153769 ... -1.76983497 -0.75038339
   1.07998198]
 [ 1.          1.23334216 -0.60516189 ...  0.39580649 -0.88267481
   0.08420343]]
Valid
[[ 1.          0.76066508  0.13245406 ... -1.76983497 -1.27954908
   1.07998198]
 [ 1.          1.23334216 -0.80185948 ...  0.11333152  0.307948
  -0.24772276]
 [ 1.         -1.13004325 -0.42129241 ...  1.99649801  0.83711369
   1.96511847]
 ...
 [ 1.         -0.18468909 -1.16318439 ... -0.26330178  0.83711369
  -0.69029101]
 [ 1.          0.76066508  2.20205476 ... -0.26330178  0.83711369
  -0.69029101]
 [ 1.         -1.60272033 -0.29301137 ... -0.26330178  0.83711369
  -0.69029101]]
Test
[

## Part 1: Linear Regression via QR Factorization

In [4]:
import math
Q = np.zeros((31715,30))
for i in range(30):
  x = train[:,i]
  for j in range(i):
    proj = (np.dot(train[:,i].transpose(),Q[:,j])/np.dot(Q[:,j].transpose(),Q[:,j])) * Q[:,j]
    x = x-proj
  Q[:,i] = x
R = np.zeros((30,30))
for i in range(30):
  for j in range(30):
    if j<i:
      R[i][j] = 0
    elif i==j:
      R[i][j] = 1
    else:
      R[i][j] = np.dot(train[:,j].transpose(), Q[:,i])/np.dot(Q[:,i],Q[:,i])
      

# print(Q)
delta = np.zeros((30,30))
for i in range(30):
  for j in range(30):
    if i==j:
      delta[i][j] = np.dot(Q[:,i],Q[:,i])
delta_inverse = np.zeros((30,30))
for i in range(30):
  for j in range(30):
    if i==j:
      delta_inverse[i][j] = 1/delta[i][j]
# print(delta_inverse)
M = np.dot(np.dot(delta_inverse,Q.transpose()),target[:31715])

W = np.ones((30))

for i in reversed(range(30)):
  m = M[i]
  for j in range(29,i,-1):
    m-=R[i][j]*W[j]
  W[i] = m
print("Weight Vector: ")
print(W)

y_predict = np.dot(test,W)
y = target[31715+3964:31715+3964+3965]
# y_p2 = np.zeros(31715)
# for i in range(30):
#   y_p2+= (np.dot(y.transpose(),Q[:,i])/np.dot(Q[:,i],Q[:,i])) * Q[:,i]
SSE = 0
TSS = 0
y_mean = np.mean(y)
for i in range(3965):
  SSE+=math.pow((y[i]-y_predict[i]),2)
  TSS+=math.pow((y[i]-y_mean),2)
print("SSE")
print(SSE)
print("TSS")
print(TSS)
R2stat = (TSS-SSE)/TSS
print("R^2")
print(R2stat)

Weight Vector: 
[ 3.40707351e+03  1.14967599e+02  4.34480546e+01  4.51110555e+02
 -2.55092374e+02  1.60750010e+02  6.92119803e+01 -2.53396335e+02
  2.09460938e+02  4.41496039e+13  3.71665242e+13  4.73826726e+13
  4.95373290e+13  4.85602957e+13  5.34979327e+02 -3.21151426e+01
 -1.55596864e+02  3.59624021e+01 -2.44067907e+02 -1.76407696e+02
  1.55632821e+00 -9.25381825e+01 -4.42548503e+01 -2.01408475e+02
  6.68095810e+01 -1.05263061e+02 -2.30367000e+01  4.45984617e+01
  6.01785590e+01  1.55678902e+02]
SSE
2.822400673836131e+28
TSS
277008716210.6508
R^2
-1.0188851500578202e+17


## Part 2: Linear Regression with Regularization

In [7]:
np.random.seed(42)
Wt = np.random.random_sample((30))
alpha = 3600
y = target[:31715]
while (True):
  deltaw = np.dot(np.dot(train.transpose(),train),Wt) + alpha*Wt - np.dot(train.transpose(),y)
  wt1 = Wt - (1e-6)*deltaw
  # wt1 = (1-(1e-6)*alpha)*Wt + np.dot((1e-6)*train.transpose(),(y-np.dot(train,Wt)))
  if (np.linalg.norm(wt1-Wt)<=.001):
    Wt = wt1
    break
  Wt = wt1
Y_val = target[31715:31715+3964]
Y_val_predict = np.dot(valid,Wt)
val_sse = 0
for i in range(3964):
  val_sse+=math.pow((Y_val[i]-Y_val_predict[i]),2)
print("Validation SSE:")
print(val_sse)
print("Weight Vector")
print(Wt)
print("Alpha")
print(alpha)

Y_test = target[31715+3964:31715+3964+3965]
Y_test_predict = np.dot(test,Wt)
Y_test_mean = np.mean(Y_test)
test_sse = 0
test_tss = 0
for i in range(3965):
  test_sse+=math.pow((Y_test[i]-Y_test_predict[i]),2)
  test_tss+=math.pow((Y_test[i]-Y_test_mean),2)
testR2 = (test_tss-test_sse)/test_tss
print("Test SSE")
print(test_sse)
print("Test TSS")
print(test_tss)
print("R^2")
print(testR2)

Validation SSE:
587126078941.055
Weight Vector
[3059.75333428   95.9369778    20.2317905   395.52581964 -209.59637407
  166.34255073   77.04010776 -249.73079823  194.63688294   71.23756594
 -156.9613368  -423.92436224  487.99709605  -28.74970653  409.27711064
   -9.6335503  -115.1697827    10.9266336  -163.33772714  -85.04563524
    5.9821681   -73.16018667  -36.86272714 -164.88538205    5.15953413
 -108.22874828   11.41421898   42.19155456   57.72331324  131.33008361]
Alpha
3600
Test SSE
270663842021.91913
Test TSS
277008716210.6508
R^2
0.022904962253630822
